### Simpler RAG Application using Langchain

###### Table of Contents

###### 1. Simpler LLM App
 -Load LLM
 -Create Prompt Tempelate
 -Merge Prompt Tempelate & LLM to create chain

###### 2. Building RAG App
-Load External Deta
-Generate Embeddings
-Build INdex in Vector Store
-Create Retreival Chain


In [ ]:
import langchain_community
from langchain_community.llms import Ollama
llm = Ollama(model="llama2") # base_url = "http://localhost:11434"
llm

In [ ]:
response = llm.invoke("Do you know abot llms?")
print(response)


LLM (Master of Laws) is a postgraduate degree that focuses on advanced legal studies. It is typically pursued by law graduates who wish to specialize in a particular area of law, such as corporate law, intellectual property law, or international law. An LLM program typically takes one year to complete and involves coursework, research, and sometimes, clinical work.

LLM programs are offered by many law schools around the world, and some universities offer specialized LLM programs in areas such as:

* Corporate and Commercial Law
* Intellectual Property Law
* International Law
* Taxation Law
* Banking and Finance Law
* Energy and Natural Resources Law
* Environmental Law
* Human Rights Law
* Labor and Employment Law
* Public International Law
* Securities Law

An LLM degree can provide several benefits, including:

1. Specialized knowledge: An LLM program allows students to specialize in a particular area of law, which can help them gain deeper insights and expertise in that field.
2. Career advancement: An LLM degree can help lawyers advance their careers by providing them with specialized knowledge and skills that can make them more attractive to potential employers.
3. Networking opportunities: LLM programs provide opportunities for students to network with other legal professionals and build connections that can be valuable in their future careers.
4. Research skills: An LLM program involves extensive research and writing, which can help students develop their research and analytical skills.
5. Global perspective: Many LLM programs offer a global perspective on law, which can help students understand the legal systems and practices of other countries and cultures.
6. Personal fulfillment: Pursuing an LLM degree can be a personally fulfilling experience, as it allows students to delve deeper into their area of interest and develop their skills and knowledge.

It's worth noting that an LLM degree is not a requirement for practicing law, and many lawyers go on to have successful careers without pursuing an LLM degree. However, for those who are interested in specializing in a particular area of law or advancing their careers, an LLM degree can be a valuable investment.

In [ ]:
response = llm.invoke("Do you know abot Claude 3?")
print(response)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an Artificial Intelligence News Reporter."),
    ("user", "{query}"),
])

In [ ]:
llm_app = prompt | llm
llm_app

In [ ]:
response = llm_app.invoke({"query":"Do you know abot Claude 3?"})
print(response)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.anthropic.com/news/claude-2-family")

docs = loader.load()

In [ ]:
urls = [
    "https://www.anthropic.com/news/releasong-claude-instant-1-2",
    "https://www.anthropic.com/news/claude-pro",
    "https://www.anthropic.com/news/claude-2",
    "https://www.anthropic.com/news/claude-2-1",
    "https://www.anthropic.com/news/claude-2-1-prompting",
    "https://www.anthropic.com/news/claude-2-3-family",
    "https://www.anthropic.com/news/claude"
]

docs = []

for url in urls:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())

len(docs)
    

In [ ]:
docs[0]

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings_llm = OllamaEmbeddings(model="llama2")
embeddings = embeddings_llm.embed_query("How are you?")
embeddings

In [ ]:
type(embeddings), len(embeddings)
embedddings = embeddings_llm.embed_documents([
    "Claude 3 is latest Conversational AI Model from Anthropic.",
    "Gemini is latest Conversational AI Model from Google",
    "Llama-2 is latest Conversational AI Model from Meta",
    "Mixtral is latest COnversational AI Model from Mistral AI",
    "GPT-4 is latest Conversational AI Model from OpenAI"
])

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(docs)

len(documents)

In [ ]:
from langchain_community.vectorstores import FAISS

vector_index = FAISS.from_documents(documents, embeddings_llm)

vector_index

In [ ]:
retreiver = vector_index.as_retriever()

relevant_docs = retreiver.invoke({"input":"Do you know about Claude 3?"})
len(relevant_docs)

In [ ]:
relevant_docs

In [ ]:
for doc in relevant_docs:
    print(f"Title: {doc.metadata['title']}, Source: {doc.metadata['source']}")

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document

prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context and your internal knowledge, Give priotiy to context and 
if you are not suree than say you are not aware of the topic:

<context>
{context}
</context>

Question: {input}
""")

document_chain = create_stuff_documents_chain(llm,prompt)

response = document_chain.invoke({
    "input" : "Do you know about Claude 3?",
    "context": [Document(page_content="Claude 3 is latest Conversational AI Model from Anthropic.")]
})

print(response)


In [ ]:
from langchain.chains import create_retrieval_chain

retreival_chain = create_retrieval_chain(retreiver, document_chain)
retreival_chain

In [ ]:
response = retreival_chain.invoke({"input": "Do you know Claude 3?"})
type(response)

In [ ]:
print(response["answer"])

In [ ]:
for doc in response["context"]:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")